# Joint Trajectory and Communication Design for Multi-UAV Enabled Wireless Networks

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings("ignore")

## Optimization Problem

\begin{align*}
\max_{\eta, \boldsymbol A, \boldsymbol Q, \boldsymbol P} \quad& \eta \tag{最大化最小平均速率}\\
s.t. \quad& \frac{1}{T} \sum_{t=1}^T \sum_{m=1}^M a_{n,m}[t] \log_2(1+\gamma_{n,m}[t]) \ge \eta \tag{最小平均速率}\\
& \sum_{n=1}^N \alpha_{n,m}[t] \le 1  \quad \forall m,t  \tag{无人机最多连 1 用户}\\
& \sum_{m=1}^M \alpha_{n,m}[t] \le 1  \quad \forall n,t  \tag{用户最多连 1 无人机}\\
& \alpha_{n,m}[t] \in \{0,1\} \tag{用户是否连接}\\
& p_m[t] \in [0, P_\max]  \tag{功率约束} \\
& q_m[1] = q_m[T]  \tag{轨迹首尾相接}\\
& \| q_m[t+1] - q_m[t]\|^2 \le s_{\max}^2 \tag{速度约束}\\
& \|q_m[t] - q_j[t] \|^2 \ge d_{\min}^2 \tag{碰撞避免}
\end{align*}

其中,
* $N$ : Number of Users
* $M$ : Number of UAVs
* $T$ : Total Duration
---
* $\eta$ : Minimum Average Communication Rate 
* $\boldsymbol A_{N×M×T}$ : User Assignment 
* $\boldsymbol Q_{M×2×T}$ : Trajectory of UAV 
* $\boldsymbol P_{N×M×T}$ : Transmit Power 

Parameters

In [4]:
numUse = 6
numUav = 3
times = 100

height = 100
Pmax = 0.1
Vmax = 50

sigma  = 10*10**(-110) * 1e-3
d0 = 1
g0 = 10*10**(-60)

## Algorithm

* Block Coodinate Descent
* Successive Convex Approximation

简化, 混合整数规划 $ \Rightarrow $ 连续变量规划

\begin{align*}
\Rightarrow \max_{\eta, \boldsymbol A, \boldsymbol Q, \boldsymbol P} \quad& \eta \\
s.t. \quad& q_{m,1} - q_{m,T} = 0  \tag{线性等式}\\
& 1 - \boldsymbol 1_N^T \boldsymbol a_{n,m,t} \ge 0  \quad \forall m,t  \tag{线性不等式}\\
& 1 - \boldsymbol 1_M^T \boldsymbol a_{n,m,t} \ge 0  \quad \forall n,t \\
& \boldsymbol A \succeq 0 \\
& \boldsymbol P \succeq 0 \\
& \boldsymbol P_\max - \boldsymbol P \succeq 0 \\
& \frac{1}{T} \sum_{t=1}^T \sum_{m=1}^M a_{n,m,t} \log_2(1+\gamma_{n,m,t}) - \eta \ge 0 \quad \forall n   \tag{非线性不等式}\\
& s_{\max}^2 - (q_{m,t+1} - q_{m,t})^T(q_{m,t+1} - q_{m,t}) \ge 0 \\
& (q_{m,t} - q_{j,t})^T(q_{m,t} - q_{j,t}) - d_{\min}^2 \ge 0 
\end{align*}

### Block Coodinate Descent

\begin{align*}
\Rightarrow \max_{\eta, \boldsymbol A} \quad& \eta  \tag{standard LP}\\
s.t. \quad& 1 - \boldsymbol 1_N^T \boldsymbol a_{n,m,t} \ge 0  \quad \forall m,t  \tag{线性不等式}\\
& 1 - \boldsymbol 1_M^T \boldsymbol a_{n,m,t} \ge 0  \quad \forall n,t \\
& \boldsymbol A \succeq 0  \\
& \frac{1}{T} \sum_{t=1}^T \sum_{m=1}^M a_{n,m,t} \log_2(1+\gamma_{n,m,t}) - \eta \ge 0 \quad \forall n
\end{align*}

---

\begin{align*}
\Rightarrow \max_{\eta, \boldsymbol Q} \quad& \eta \\
s.t. \quad& q_{m,1} - q_{m,T} = 0  \tag{线性等式}\\
& \frac{1}{T} \sum_{t=1}^T \sum_{m=1}^M a_{n,m,t} \log_2(1+\gamma_{n,m,t}) - \eta \ge 0 \quad \forall n   \tag{非线性不等式}\\
& s_{\max}^2 - (q_{m,t+1} - q_{m,t})^T(q_{m,t+1} - q_{m,t}) \ge 0 \\
& (q_{m,t} - q_{j,t})^T(q_{m,t} - q_{j,t}) - d_{\min}^2 \ge 0 
\end{align*}

---

\begin{align*}
\Rightarrow \max_{\eta, \boldsymbol P} \quad& \eta\\
s.t. \quad& \boldsymbol P \succeq 0  \tag{线性不等式}\\
& \boldsymbol P_\max - \boldsymbol P \succeq 0 \\
& \frac{1}{T} \sum_{t=1}^T \sum_{m=1}^M a_{n,m,t} \log_2(1+\gamma_{n,m,t}) - \eta \ge 0 \quad \forall n   \tag{非线性不等式}\\
\end{align*}

### Successive Convex Approximation